In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import math
import os
import dpsimpy
import glob
import requests

In [ ]:
sim_name = "DP_WSCC9bus_SGReducedOrderVBR"
final_time = 0.1
time_step = 10e-6
with_fault = True
start_time_fault = 0.2
end_time_fault = 0.3
sg_type = "4"
fault_bus_name = "BUS6"
intertia_scaling_factor = 1.0
log_dir = "logs"
solver = dpsimpy.DirectLinearSolverImpl.SparseLU
sim_name_pf = sim_name + "_PF"
log_level = dpsimpy.LogLevel.info

if time_step < 10e-6:
    log_down_sampling = floor((10e-6) / time_step)
else:
    log_down_sampling = 1.0

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

filename = 'WSCC-09_RX'
url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/WSCC-09/WSCC-09_RX_Dyn_Full/WSCC-09_RX'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')
download_grid_data(filename+'_DI.xml', url+'_DI.xml')
    
files = glob.glob(filename+'_*.xml')

dpsimpy.Logger.set_log_dir(log_dir+"/"+sim_name_pf)
reader = dpsimpy.CIMReader(sim_name_pf, log_level, log_level)

system_pf = reader.loadCIM(60, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
system_pf.component('GEN1').modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)
logger_pf = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger_pf.log_attribute(node.name() + ".V", node.attr("v"))
    
sim_pf = dpsimpy.Simulation(sim_name_pf, log_level)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(final_time)
sim_pf.set_final_time(2*final_time)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(True)
sim_pf.add_logger(logger_pf)
sim_pf.run()

In [ ]:
## Dynamic Simulation
def run_simulation(fillin, btf, partial):
    dpsimpy.Logger.set_log_dir(log_dir+"/"+sim_name)
    reader2 = dpsimpy.CIMReader(sim_name, log_level, log_level)
    if sg_type == "3":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG3OrderVBR)
    elif sg_type == "4":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG4OrderVBR)
    elif sg_type == "6b":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG6bOrderVBR)
    else:
        raise Exception("Unsupported reduced-order SG type!")

    fault_dp = dpsimpy.sp.ph1.Switch("Fault", log_level)

    if with_fault == True:
        n1_dp = dpsimpy.dp.SimNode(fault_bus_name, dpsimpy.PhaseType.Single)
        fault_dp.set_parameters(1e12, 0.02*529)
        fault_dp.connect([dpsimpy.dp.SimNode.gnd, n1_dp])
        fault_dp.open()
        sys.add(fault_dp)

    sys.init_with_powerflow(system_pf)
    for comp in sys.components:
       # I assume this is how you access the generators
       if "GEN" in comp.name():
           gen_pf = system_pf.component(comp.name())
           comp.get_terminal(index=0).set_power(- gen_pf.get_apparent_power())
           comp.scale_inertia_constant(intertia_scaling_factor)
           comp.set_model_as_current_source(False)

    logger = dpsimpy.Logger(sim_name)#, True, log_down_sampling)
    for node in sys.nodes:
        logger.log_attribute(node.name() + ".V", node.attr("v"))

    for comp in sys.components:
        if "GEN" in comp.name():
            logger.log_attribute(comp.name() + ".Tm", comp.attr("Tm"))
            logger.log_attribute(comp.name() + ".Te", comp.attr("Te"))
            logger.log_attribute(comp.name() + ".omega", comp.attr("w_r"))
            logger.log_attribute(comp.name() + ".delta", comp.attr("delta"))
            
    config = dpsimpy.DirectLinearSolverConfiguration()
    config.set_btf(btf)
    config.set_fill_in_reduction_method(fillin)
    config.set_partial_refactorization_method(partial)
    implementation = dpsimpy.DirectLinearSolverImpl.KLU

    sim = dpsimpy.Simulation(sim_name, log_level)
    sim.set_system(sys)
    sim.set_domain(dpsimpy.Domain.DP)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_time_step(time_step)
    sim.set_final_time(final_time)
    sim.do_system_matrix_recomputation(True)
    sim.set_direct_solver_implementation(implementation)
    sim.set_direct_linear_solver_configuration(config)
    sim.add_logger(logger)

    if with_fault:
        fault_event1 = dpsimpy.event.SwitchEvent(start_time_fault, fault_dp, True)
        fault_event2 = dpsimpy.event.SwitchEvent(end_time_fault, fault_dp, False)
        sim.add_event(fault_event1)
        sim.add_event(fault_event2)

    sim.run()
    sim.log_lu_times()

In [ ]:
%%capture
fillin = dpsimpy.fill_in_reduction_method.amd
btf = dpsimpy.use_btf.do_btf
partial = dpsimpy.partial_refactorization_method.factorization_path

run_simulation(fillin, btf, partial)
ts_dpsim_lv_amd = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_LeftVector.csv")
ts_dpsim_rv_amd = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_RightVector.csv")

In [ ]:
%%capture
fillin = dpsimpy.fill_in_reduction_method.amd_nv
btf = dpsimpy.use_btf.do_btf
partial = dpsimpy.partial_refactorization_method.factorization_path

run_simulation(fillin, btf, partial)
ts_dpsim_lv_amd_nv = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_LeftVector.csv")
ts_dpsim_rv_amd_nv = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_RightVector.csv")

In [ ]:
%%capture
fillin = dpsimpy.fill_in_reduction_method.amd_ra
btf = dpsimpy.use_btf.do_btf
partial = dpsimpy.partial_refactorization_method.refactorization_restart

run_simulation(fillin, btf, partial)
ts_dpsim_lv_amd_ra = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_LeftVector.csv")
ts_dpsim_rv_amd_ra = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_RightVector.csv")

In [ ]:
tolerance = 1e-4

for entry in ts_dpsim_rv_amd:
    amd_values = ts_dpsim_rv_amd[entry].values
    amd_nv_values = ts_dpsim_rv_amd_nv[entry].values
    amd_ra_values = ts_dpsim_rv_amd_ra[entry].values
    # 2-norm of errors
    error_1 = np.linalg.norm(amd_values-amd_nv_values, ord=2)
    error_2 = np.linalg.norm(amd_ra_values-amd_values, ord=2)
    assert(error_1 < tolerance)
    assert(error_2 < tolerance)

for entry in ts_dpsim_lv_amd:
    amd_values = ts_dpsim_lv_amd[entry].values
    amd_nv_values = ts_dpsim_lv_amd_nv[entry].values
    amd_ra_values = ts_dpsim_lv_amd_ra[entry].values
    # 2-norm of errors
    error_1 = np.linalg.norm(amd_values-amd_nv_values, ord=2)
    error_2 = np.linalg.norm(amd_ra_values-amd_values, ord=2)
    assert(error_1 < tolerance)
    assert(error_2 < tolerance)